# House price prediction

In [69]:
import numpy as np 
import pandas as pd 

## Glimpse of the data

In [70]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
train.shape, test.shape

((1460, 81), (1459, 80))

In [71]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [72]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [73]:
set(train.dtypes.to_list())

{dtype('int64'), dtype('float64'), dtype('O')}

In [74]:
train.select_dtypes(include=['int64', 'float64']).head()


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [75]:
train.select_dtypes(include=['object']).head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


Missing Data

In [76]:
missing_count = train.isnull().sum().sort_values(ascending=False)
missing_percent = train.isnull().mean().sort_values(ascending=False)
missing_data = pd.concat([missing_count, missing_percent], axis=1, keys=['Count', 'Percent'])
missing_data.head(20)

,Count,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479
GarageFinish,81,0.055479


In [77]:
# remove columns with missing data more than 60%
large_missing_columns = missing_data[missing_data['Percent'] > 0.6].index
large_missing_columns

Index(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], dtype='object')

In [78]:
large_missing_columns.to_list()

['PoolQC', 'MiscFeature', 'Alley', 'Fence']

## Tiền xử lý

In [79]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

# Tách cột Id ra riêng phòng trường hợp sử dụng khi cần thiết
train_ID = train['Id']
test_ID = test['Id']

# Xóa cột 'Id' trong tập dữ liệu train và test
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

# Chúng ta sẽ kết hợp 2 tập dữ liệu train và test ở các bước sau để xử lý
# tiền dữ liệu nên phải cần lưu lại số lượng dữ liệu ở mỗi tập để tách chúng
# trở lại
ntrain = train.shape[0]
ntest = test.shape[0]

y = train.SalePrice.values

# all_data được sử dụng để xử lý data ở bước 2:
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)

### Missing Data

Trong bước này chưa nhất thiết phải remove dữ liệu missing data cao, vì sau bước tiền xử lý chúng ta còn bước feature selection, tại đây thì những dữ liệu thiếu nhiều sẽ bị loại bỏ một cách tự động.

In [80]:
all_data_nomissing = all_data.copy()

In [83]:
# Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
# Điền dữ liệu bị thiếu bằng giá trị median
all_data_nomissing["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

# Điền dữ liệu bị thiếu bằng giá trị 'Typ'
all_data_nomissing["Functional"] = all_data['Functional'].fillna("Typ")

# 
fake_nummeric_cols = ['MSSubClass', 'OverallQual', 'OverallCond', 'YrSold', 'MoSold']
for col in fake_nummeric_cols:
    all_data_nomissing[col] = all_data[col].astype(str)

In [85]:
# This group will be filled NaNs by 'None'
missing_cols_group1 = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 
                       'FireplaceQu', 'MSSubClass', 'GarageType', 
                       'GarageFinish', 'GarageQual', 'GarageCond', 
                       'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                       'BsmtFinType1', 'BsmtFinType2', 'MasVnrType']

# This group will be filled NaNs by 0
missing_cols_group2 = ['GarageYrBlt', 'GarageArea', 'GarageCars', 
                       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 
                       'MasVnrArea']

# This group will be filled NaNs by mode
missing_cols_group3 = ['MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType', 'Utilities']
